PULLING IN WET FILES FROM COMMONCRAWL AND CREATING OUR INITIAL DATASET

In [ ]:
import requests
import gzip
import os
import warcio
from warcio.archiveiterator import ArchiveIterator

# List of URLs of WET files you want to download
wet_file_urls = [
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198861173.16/wet/CC-MAIN-20240612140424-20240612170424-00000.warc.wet.gz',
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198861578.89/wet/CC-MAIN-20240614235857-20240615025857-00080.warc.wet.gz',
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198861583.78/wet/CC-MAIN-20240615031115-20240615061115-00117.warc.wet.gz',
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198861584.65/wet/CC-MAIN-20240615062230-20240615092230-00235.warc.wet.gz',
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198861657.69/wet/CC-MAIN-20240616074847-20240616104847-00289.warc.wet.gz',
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198861665.97/wet/CC-MAIN-20240616141113-20240616171113-00744.warc.wet.gz',
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198861670.48/wet/CC-MAIN-20240616172129-20240616202129-00068.warc.wet.gz',
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198861701.67/wet/CC-MAIN-20240617091230-20240617121230-00468.warc.wet.gz',
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198861762.73/wet/CC-MAIN-20240618140737-20240618170737-00599.warc.wet.gz',
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198861794.76/wet/CC-MAIN-20240618203026-20240618233026-00070.warc.wet.gz',
    'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-26/segments/1718198862404.32/wet/CC-MAIN-20240622144011-20240622174011-00455.warc.wet.gz',
    ]

# Directory to save downloaded WET files
download_dir = 'wet_files'

# Create the directory if it doesn't exist
os.makedirs(download_dir, exist_ok=True)

# Download each WET file
for url in wet_file_urls:
    filename = os.path.join(download_dir, url.split('/')[-1])
    print(f'Downloading {filename}...')
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f'{filename} downloaded successfully.')
    except requests.exceptions.RequestException as e:
        print(f'Failed to download {filename}. Error: {e}')

wet_files\CC-MAIN-20240612140424-20240612170424-00000.warc.wet.gz downloaded successfully.
wet_files\CC-MAIN-20240614235857-20240615025857-00080.warc.wet.gz downloaded successfully.
wet_files\CC-MAIN-20240615031115-20240615061115-00117.warc.wet.gz downloaded successfully.
wet_files\CC-MAIN-20240615062230-20240615092230-00235.warc.wet.gz downloaded successfully.
wet_files\CC-MAIN-20240616074847-20240616104847-00289.warc.wet.gz downloaded successfully.
wet_files\CC-MAIN-20240616141113-20240616171113-00744.warc.wet.gz downloaded successfully.
wet_files\CC-MAIN-20240616172129-20240616202129-00068.warc.wet.gz downloaded successfully.
wet_files\CC-MAIN-20240617091230-20240617121230-00468.warc.wet.gz downloaded successfully.
wet_files\CC-MAIN-20240618140737-20240618170737-00599.warc.wet.gz downloaded successfully.
wet_files\CC-MAIN-20240618203026-20240618233026-00070.warc.wet.gz downloaded successfully.
wet_files\CC-MAIN-20240622144011-20240622174011-00455.warc.wet.gz downloaded successfully.

In [ ]:
import pandas as pd

# Directory where the WET files are saved
download_dir = 'wet_files'

# List to store the parsed data
data = []

# Iterate through each WET file in the directory
for filename in os.listdir(download_dir):
    if filename.endswith('.warc.wet.gz'):
        file_path = os.path.join(download_dir, filename)
        print(f'Processing {file_path}...')

        with gzip.open(file_path, 'rb') as f:
            for record in ArchiveIterator(f):
                if record.rec_type == 'conversion':
                    content = record.content_stream().read().decode('utf-8')
                    # Example: Extracting URL and text content
                    url = record.rec_headers.get_header('WARC-Target-URI')
                    language = record.rec_headers.get_header('WARC-Identified-Content-Language')
                    data.append({'url': url, 'content': content, 'language' : language})

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)

Processing wet_files\CC-MAIN-20240612140424-20240612170424-00000.warc.wet.gz...
Processing wet_files\CC-MAIN-20240614235857-20240615025857-00080.warc.wet.gz...
Processing wet_files\CC-MAIN-20240615031115-20240615061115-00117.warc.wet.gz...
Processing wet_files\CC-MAIN-20240615062230-20240615092230-00235.warc.wet.gz...
Processing wet_files\CC-MAIN-20240616074847-20240616104847-00289.warc.wet.gz...
Processing wet_files\CC-MAIN-20240616141113-20240616171113-00744.warc.wet.gz...
Processing wet_files\CC-MAIN-20240616172129-20240616202129-00068.warc.wet.gz...
Processing wet_files\CC-MAIN-20240617091230-20240617121230-00468.warc.wet.gz...
Processing wet_files\CC-MAIN-20240618140737-20240618170737-00599.warc.wet.gz...
Processing wet_files\CC-MAIN-20240618203026-20240618233026-00070.warc.wet.gz...
Processing wet_files\CC-MAIN-20240622144011-20240622174011-00455.warc.wet.gz...


In [ ]:
# remove non-English web pages
df = df[df['language']=='eng']
df

,url,content,language
9,http://10kh.show/00045-season-5-opener-the-100...,"The Ten Thousand (10,000) Heroes Show: #00045 ...",eng
10,http://10kh.show/website/2022/04,"The Ten Thousand (10,000) Heroes Show\nToggle ...",eng
13,http://122colmorerow.co.uk/,Birmingham Office Space | 122 Colmore Row\nHOM...,eng
14,http://123495479.indirettamente.it/2500-net-ma...,indirettamente.it\nInquire about this domain\n...,eng
15,http://123495479.indirettamente.it/kurdele-%C3...,indirettamente.it\nInquire about this domain\n...,eng
...,...,...,...
336530,https://zoom.earth/places/pakistan/panjgur/,"Panjgur, Pakistan | Live Weather Satellite Map...",eng
336532,https://zorion.com/stocks/matson-stock-price/,How to Invest in Matson Inc Stock — Buy MATX S...,eng
336534,https://zouchmagazine.com/tag/accessories/,accessories Archives - Zouch - Zouch\nMasthead...,eng
336542,https://zsfglearn.org/home-2/who-we-are/,Who We Are – Department of Education & Trainin...,eng


In [ ]:

#drop language column and seperate out URL's to run through the scraper
df = df.drop(columns=["language"])



In [ ]:
# Define a regular expression pattern to match all special characters
pattern = r'[^a-zA-Z0-9\s]'

# Replace special characters with an empty string
df['content'] = df['content'].str.replace(pattern, '', regex=True)

# Remove newline characters
df['content'] = df['content'].str.replace(r'\n', '', regex=True)

df= df.applymap(lambda x: x.lower() if isinstance(x, str) else x) #lowercase

print(df)

                                                      url  \
9       http://10kh.show/00045-season-5-opener-the-100...   
10                       http://10kh.show/website/2022/04   
13                            http://122colmorerow.co.uk/   
14      http://123495479.indirettamente.it/2500-net-ma...   
15      http://123495479.indirettamente.it/kurdele-%c3...   
...                                                   ...   
336530        https://zoom.earth/places/pakistan/panjgur/   
336532      https://zorion.com/stocks/matson-stock-price/   
336534         https://zouchmagazine.com/tag/accessories/   
336542           https://zsfglearn.org/home-2/who-we-are/   
336550             https://zxdemo.org/productions/336067/   

                                                  content  contains_keywords  
9       the ten thousand 10000 heroes show 00045 seaso...              False  
10      the ten thousand 10000 heroes showtoggle navig...              False  
13      birmingham office spac

Key Word Searching

In [ ]:
#key words

keywords = ['census', 'uscb', 'american community survey','american','nih','government','survey','population']

# Function to check if any keywords are in the content
def contains_keywords(content, keywords):
    return any(keyword in content for keyword in keywords)

# Apply the function to create a new column
df['contains_keywords'] = df['content'].apply(lambda x: contains_keywords(x, keywords))

print(df)

                                                      url  \
9       http://10kh.show/00045-season-5-opener-the-100...   
10                       http://10kh.show/website/2022/04   
13                            http://122colmorerow.co.uk/   
14      http://123495479.indirettamente.it/2500-net-ma...   
15      http://123495479.indirettamente.it/kurdele-%c3...   
...                                                   ...   
336530        https://zoom.earth/places/pakistan/panjgur/   
336532      https://zorion.com/stocks/matson-stock-price/   
336534         https://zouchmagazine.com/tag/accessories/   
336542           https://zsfglearn.org/home-2/who-we-are/   
336550             https://zxdemo.org/productions/336067/   

                                                  content  contains_keywords  
9       the ten thousand 10000 heroes show 00045 seaso...              False  
10      the ten thousand 10000 heroes showtoggle navig...              False  
13      birmingham office spac

In [ ]:
keywords_present_df = df[df['contains_keywords']==1]
keywords_present_df

,url,content,contains_keywords
69,http://925-www.trustlink.org/review/american-p...,michael hs review of american publisher housem...,True
107,http://aipaea09.blogspot.com/2010/03/national-...,all india postal accounts employees associatio...,True
119,http://almagor.blogspot.com/2005_03_20_archive...,israel democracy human rights politics and soc...,True
154,http://appendx.org/issue1/fields/index4.htm,appendx 1 a black manifesto darell w fieldsa...,True
161,http://archeryenm734.timeforchangecounselling....,7 simple secrets to totally rocking your lol...,True
...,...,...,...
336433,https://zacsint.com/using-your-poland-women-to...,using your poland women to create more money ...,True
336440,https://zambia.accessbankplc.com/about-us/boar...,access bank zambia access bank zambiaabout us...,True
336443,https://zanerpreciousmetals.com/jewelry-2,zaner precious metals products tagged with jew...,True
336444,https://zanerpreciousmetals.com/zaner-daily-pr...,zaner precious metals zaner daily precious met...,True


Web Scraper

In [ ]:
seedlinks = keywords_present_df['url'].tolist()

len(seedlinks)

29361

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


# List of URLs to process

urls = seedlinks[1:29362]


# Compile the regex pattern
pattern = re.compile(r'.census.gov.*')

# Initialize an empty list to store all data
all_data = []

# Iterate over each URL in the list
for url in urls:
    try:
        # Attempt to fetch the content from the URL
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all links matching the pattern
        links = [link.get('href') for link in soup.find_all('a', href=pattern)]

        if links:
            all_data.extend([(url, link) for link in links])
        else:
            all_data.append((url, 'NONE'))

    except requests.exceptions.RequestException as e:
        # Handle any exception raised during the request
        print(f"Failed to scrape {url}: {e}")
        # Add a failure entry to the data
        all_data.append((url, 'ERROR'))

# Create a DataFrame with the seed links and the found links
scrapedlinks = pd.DataFrame(all_data, columns=['Seed Link', 'Found Links'])
print(scrapedlinks)

In [ ]:
#combining original common crawl data
scrapedlinks.rename(columns={'Seed Link': 'url'}, inplace=True)
FullDataSet = pd.merge(scrapedlinks,keywords_present_df,on='url', how="inner")
FullDataSet

,url,Found Links,content,contains_keywords
0,http://aipaea09.blogspot.com/2010/03/national-...,NONE,all india postal accounts employees associatio...,True
1,http://almagor.blogspot.com/2005_03_20_archive...,NONE,israel democracy human rights politics and soc...,True
2,http://appendx.org/issue1/fields/index4.htm,NONE,appendx 1 a black manifesto darell w fieldsa...,True
3,http://archeryenm734.timeforchangecounselling....,NONE,7 simple secrets to totally rocking your lol...,True
4,http://archive.wn.com/2005/03/10/1400/jazz/,NONE,wednesday 09 march 2005this page was original...,True
...,...,...,...,...
29385,https://zacsint.com/using-your-poland-women-to...,ERROR,using your poland women to create more money ...,True
29386,https://zambia.accessbankplc.com/about-us/boar...,NONE,access bank zambia access bank zambiaabout us...,True
29387,https://zanerpreciousmetals.com/jewelry-2,ERROR,zaner precious metals products tagged with jew...,True
29388,https://zanerpreciousmetals.com/zaner-daily-pr...,ERROR,zaner precious metals zaner daily precious met...,True


In [ ]:
#output links from web scraper so we don't have to scrape them repeatedly
FullDataSet.to_excel('scrapedlinks.xlsx', index=False)